# Preprocess for analysis

## Import Packages

In [35]:
import pandas as pd
import numpy as np
import tabulate as tb

import wrds

from statsmodels.regression.rolling import RollingOLS
from statsmodels.tsa.ar_model import AutoReg
from scipy.stats import skew, kurtosis, rankdata
import statsmodels.api as sm

In [6]:
# For testing 
import time

In [2]:
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


# Beta

According to the CAPM, each expected stock return is equal to the risk-free return plus the market beta multiplied by the market risk premium.   
Thus,
$$E[R_{i,t}] = R_{f,t} + \beta_i (E[R_{m,t}] - R_{f,t})$$
where, beta of security $i$ is 
$$\beta_i = \frac{Cov(R_{i,t}, R_{m,t})}{Var(R_{m,t})}$$
where, $R_{i,t}, R_{m,t}$ is the return of security $i$ and market portfolio

There are several empirically testable predictions.   
In this chapter we show two of them.

1. The CAPM predicts that cross-sectional variation in expected returns is driven by variation in the security's beta.   
    
    1.1 Test the cross-sectional ability of beta to predict future excess returns.   
        
    Fama and MacBeth (1973) finds the positive correlation between excess returns and the beta, but recent studies (Fama and French 1992; 1993, and others) fail to find the correlation.
    
    1.2 Test cross-sectional ability of other variables to predict future excess returns.
        
    We call these variables as "anomalies".


2. Average excess returns, after accounting for effect of beta, should be zero.   

    We can find this evidence by examining the intercept ($\alpha$) term of cross-sectional regressions of security excess returns on estimates of beta.  
     
    If CAPM can predict stock returns, intercept should be 0.   

# 1. Estimating beta

$r_{i,t} = \alpha_i + \beta_i MKT_t + \epsilon_{i,t}$   

where $r_{i,t}$ and $MKT_t$ is the excess returns of stock $i$ and market portfolio(market factor) during $t$.   
   
   
in CAPM, $\alpha_i$ should be zero.   

in Python code   

```python
r_it = alpha_i + beta_i * MKT_it + epsilon_it
```

There is a difference in the period and length of data used to estimate market beta.   

Most commonly, market beta during $t$ is estimated using only data up to and including $t$.   

The most popular beta estimates cover $t-11$ to $t$ and estimate $t+1$.   

For the quality of the estimate, it is reasonable to fit using at least 200 data points over a 12-month period.   

Another common approach is to use the excess returns of monthly returns over 5 years.

In this chapter, using 9 combination of estimation periods and data frequency.   

Five using daily data, 1M, 3M, 6M, 12M, and 24M, require 15days, 50 days, 100days, 200days and 450 days for valid return data.   

We denote these estimates of market beta, caculated using daily excess return data, as $\beta^{kM}$, where $k \in \{1,3,6,12,24\}$.   

The daily stock return data come from __crsp.dsf__, and daily MKT factor returns and dailiy risk free rates come from Kenneth French's data library.

#### Data from WRDS

In [4]:
# Data description for the CRSP daily stock file
conn.describe_table(library='crsp', table='dsf')

Approximately 105258384 rows in crsp.dsf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),CUSIP Header
1,permno,True,INTEGER,Permanent Security Number
2,permco,True,INTEGER,Permanent Company Number
3,issuno,True,INTEGER,Nasdaq Issue Number
4,hexcd,True,SMALLINT,Exchange Code Header
5,hsiccd,True,INTEGER,Standard Industrial Classification Code
6,date,True,DATE,Date of Observation
7,bidlo,True,"NUMERIC(11, 5)",Bid or Low Price
8,askhi,True,"NUMERIC(11, 5)",Ask or High Price
9,prc,True,"NUMERIC(11, 5)",Price or Bid/Ask Average


In [5]:
# Data description for fama french factors daily
conn.describe_table(library='ff', table='factors_daily')

Approximately 25773 rows in ff.factors_daily.


,name,nullable,type,comment
0,date,True,DATE,None
1,mktrf,True,"NUMERIC(8, 6)",Excess Return on the Market
2,smb,True,"NUMERIC(8, 6)",Small-Minus-Big Return
3,hml,True,"NUMERIC(8, 6)",High-Minus-Low Return
4,rf,True,"NUMERIC(7, 5)",Risk-Free Return Rate (One Month Treasury Bill...
5,umd,True,"NUMERIC(8, 6)",Momentum


In [95]:
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [96]:
# Download the data from the CRSP daily stock file
crsp_d = conn.raw_sql("""
                      SELECT permno, date, ret, retx, prc, shrout
                      FROM crsp.dsf
                      WHERE date <= '2012-12-31'
                      """)

In [100]:
crsp_d.to_csv("C:/Users/YeonChan Kang/Desktop/Local_repo/EAP_CSSR_Tutorial/data/crsp_d.csv", index=False)

In [98]:
crsp_d

,permno,date,ret,retx,prc,shrout
0,10000,1986-01-07,NaN,NaN,-2.5625,3680.0
1,10000,1986-01-08,-0.024390,-0.024390,-2.5000,3680.0
2,10000,1986-01-09,0.000000,0.000000,-2.5000,3680.0
3,10000,1986-01-10,0.000000,0.000000,-2.5000,3680.0
4,10000,1986-01-13,0.050000,0.050000,-2.6250,3680.0
...,...,...,...,...,...,...
143732,93436,2012-12-24,0.008235,0.008235,34.2800,113779.0
143733,93436,2012-12-26,-0.020128,-0.020128,33.5900,113779.0
143734,93436,2012-12-27,0.002977,0.002977,33.6900,113779.0
143735,93436,2012-12-28,-0.013951,-0.013951,33.2200,113779.0


In [ ]:
# import the data from local
crsp_d = pd.read_csv("C:/Users/YeonChan Kang/Desktop/Local_repo/EAP_CSSR_Tutorial/data/crsp_d.csv", index_col=0)

In [101]:
# Convert the date column to datetime
crsp_d['date'] = pd.to_datetime(crsp_d['date'])

In [102]:
crsp_d

,permno,date,ret,retx,prc,shrout
0,10000,1986-01-07,NaN,NaN,-2.5625,3680.0
1,10000,1986-01-08,-0.024390,-0.024390,-2.5000,3680.0
2,10000,1986-01-09,0.000000,0.000000,-2.5000,3680.0
3,10000,1986-01-10,0.000000,0.000000,-2.5000,3680.0
4,10000,1986-01-13,0.050000,0.050000,-2.6250,3680.0
...,...,...,...,...,...,...
143732,93436,2012-12-24,0.008235,0.008235,34.2800,113779.0
143733,93436,2012-12-26,-0.020128,-0.020128,33.5900,113779.0
143734,93436,2012-12-27,0.002977,0.002977,33.6900,113779.0
143735,93436,2012-12-28,-0.013951,-0.013951,33.2200,113779.0


In [103]:
# Download the data from fama french factors daily
ff_d = conn.raw_sql("""
                    SELECT date, mktrf, rf
                    FROM ff.factors_daily
                    WHERE date <= '2012-12-31'
                    """)

In [104]:
ff_d['date'] = pd.to_datetime(ff_d['date'])

In [105]:
ff_d

,date,mktrf,rf
0,1926-07-01,0.0010,0.00009
1,1926-07-02,0.0045,0.00009
2,1926-07-06,0.0017,0.00009
3,1926-07-07,0.0009,0.00009
4,1926-07-08,0.0021,0.00009
...,...,...,...
22876,2012-12-24,-0.0024,0.00001
22877,2012-12-26,-0.0054,0.00001
22878,2012-12-27,-0.0011,0.00001
22879,2012-12-28,-0.0100,0.00001


In [106]:
# Merge daily dataset
daily_data = pd.merge(crsp_d, ff_d, on='date', how='left')

In [107]:
daily_data

,permno,date,ret,retx,prc,shrout,mktrf,rf
0,10000,1986-01-07,NaN,NaN,-2.5625,3680.0,0.0138,0.00025
1,10000,1986-01-08,-0.024390,-0.024390,-2.5000,3680.0,-0.0216,0.00025
2,10000,1986-01-09,0.000000,0.000000,-2.5000,3680.0,-0.0117,0.00025
3,10000,1986-01-10,0.000000,0.000000,-2.5000,3680.0,-0.0002,0.00025
4,10000,1986-01-13,0.050000,0.050000,-2.6250,3680.0,0.0028,0.00025
...,...,...,...,...,...,...,...,...
83643732,93436,2012-12-24,0.008235,0.008235,34.2800,113779.0,-0.0024,0.00001
83643733,93436,2012-12-26,-0.020128,-0.020128,33.5900,113779.0,-0.0054,0.00001
83643734,93436,2012-12-27,0.002977,0.002977,33.6900,113779.0,-0.0011,0.00001
83643735,93436,2012-12-28,-0.013951,-0.013951,33.2200,113779.0,-0.0100,0.00001


In [108]:
# Calculate excess return
daily_data['exret'] = daily_data['ret'] - daily_data['rf']

In [109]:
daily_data

,permno,date,ret,retx,prc,shrout,mktrf,rf,exret
0,10000,1986-01-07,NaN,NaN,-2.5625,3680.0,0.0138,0.00025,NaN
1,10000,1986-01-08,-0.024390,-0.024390,-2.5000,3680.0,-0.0216,0.00025,-0.024640
2,10000,1986-01-09,0.000000,0.000000,-2.5000,3680.0,-0.0117,0.00025,-0.000250
3,10000,1986-01-10,0.000000,0.000000,-2.5000,3680.0,-0.0002,0.00025,-0.000250
4,10000,1986-01-13,0.050000,0.050000,-2.6250,3680.0,0.0028,0.00025,0.049750
...,...,...,...,...,...,...,...,...,...
83643732,93436,2012-12-24,0.008235,0.008235,34.2800,113779.0,-0.0024,0.00001,0.008225
83643733,93436,2012-12-26,-0.020128,-0.020128,33.5900,113779.0,-0.0054,0.00001,-0.020138
83643734,93436,2012-12-27,0.002977,0.002977,33.6900,113779.0,-0.0011,0.00001,0.002967
83643735,93436,2012-12-28,-0.013951,-0.013951,33.2200,113779.0,-0.0100,0.00001,-0.013961


##### using the package function from nafitools

In [71]:
# Later Updated

#### Cleansing data

Handling Missing data

When we use CRSP and K French library data, we can just drop the missing values.   

For more on Missing data, see [[url]]

In [110]:
len(daily_data)

83643737

In [111]:
daily_data.isna().sum()

permno          0
date            0
ret       1741702
retx      1741702
prc       1708339
shrout          0
mktrf       79472
rf          79472
exret     1818524
dtype: int64

In [112]:
daily_data.dropna(inplace=True)

In [113]:
len(daily_data)

81825213

In [114]:
daily_data.isna().sum()

permno    0
date      0
ret       0
retx      0
prc       0
shrout    0
mktrf     0
rf        0
exret     0
dtype: int64

PRC adjust

If we use the CRSP price, apply the absolute value to the price.

In [115]:
daily_data['prc'] = daily_data['prc'].abs()

In [116]:
# Check the minimum price is greater than 0
daily_data['prc'].min()

0.0039

In [117]:
daily_data

,permno,date,ret,retx,prc,shrout,mktrf,rf,exret
1,10000,1986-01-08,-0.024390,-0.024390,2.500,3680.0,-0.0216,0.00025,-0.024640
2,10000,1986-01-09,0.000000,0.000000,2.500,3680.0,-0.0117,0.00025,-0.000250
3,10000,1986-01-10,0.000000,0.000000,2.500,3680.0,-0.0002,0.00025,-0.000250
4,10000,1986-01-13,0.050000,0.050000,2.625,3680.0,0.0028,0.00025,0.049750
5,10000,1986-01-14,0.047619,0.047619,2.750,3680.0,0.0001,0.00025,0.047369
...,...,...,...,...,...,...,...,...,...
83643732,93436,2012-12-24,0.008235,0.008235,34.280,113779.0,-0.0024,0.00001,0.008225
83643733,93436,2012-12-26,-0.020128,-0.020128,33.590,113779.0,-0.0054,0.00001,-0.020138
83643734,93436,2012-12-27,0.002977,0.002977,33.690,113779.0,-0.0011,0.00001,0.002967
83643735,93436,2012-12-28,-0.013951,-0.013951,33.220,113779.0,-0.0100,0.00001,-0.013961


In this chapter we will use Winsorize.

For more on Winsorize, see Chap. 1. Preliminaries.

In [118]:
def winsorize(data, col=False, except_col=['permno', 'date'], lower_bound=0.01, upper_bound=0.99):
    """
    Winsorizes the input data by replacing extreme values with the nearest values within the specified quantiles.
    if col = "column A", the function will only winsorize the specified "column A".

    Parameters:
    data (pd.Series or pd.DataFrame): The data to be winsorized.
    column (str or bool): The column to be winsorized or False to winsorize all columns. Defaults to False.
    except_col (str or bool): The column to be excluded from winsorization or False to winsorize all columns. Defaults to ['permno', 'date'].
                                For CRSP, exclude the permno and date columns. Otherwise, specify the id and date columns that match your data.
    lower (float): The lower quantile threshold. Defaults to 0.01.
    upper (float): The upper quantile threshold. Defaults to 0.99.
    copy (bool): Whether to return a copy of the data or to modify it in place. Defaults to True.
    
    Returns:
        pd.Series or pd.DataFrame: The winsorized data.
    """
    
    if except_col:
        except_df = data[except_col]
        data = data.drop(columns=except_col)
    
    else:
        print("There are no columns to be excluded")
    
    if col:
        data = data[col]

        print(f"Original maximum of {col}: {data.max()} and Original minimum of {col}: {data.min()}\n")

        qtl = data.quantile([lower_bound, upper_bound])
        
        # Winsorize the data
        data = np.where(data < qtl.loc[lower_bound], qtl.loc[lower_bound], data)
        data = np.where(data > qtl.loc[upper_bound], qtl.loc[upper_bound], data)
        
        print(f"New maximum of {col}: {data.max()} and New minimum of {col}: {data.min()}\n \n")
    
    else:
        for col in data.columns:
            col_data = data[col]
            
            print(f"Original maximum of {col}: {col_data.max()} and Original minimum of {col}: {col_data.min()}\n")
            
            qtl = col_data.quantile([lower_bound, upper_bound])
            
            # Winsorize the data
            col_data = np.where(col_data < qtl.loc[lower_bound], qtl.loc[lower_bound], col_data)
            col_data = np.where(col_data > qtl.loc[upper_bound], qtl.loc[upper_bound], col_data)
            
            print(f"New maximum of {col}: {col_data.max()} and New minimum of {col}: {col_data.min()}\n \n")
            
            data[col] = col_data
    
    if except_col:
        data = pd.concat([except_df, data], axis=1)
    
    return data
    

In [119]:
# French data is already in clean format so we will only winsorize the CRSP data
# shrout is also no need to be winsorized since it is the number of shares outstanding
daily_data = winsorize(data=daily_data, col=False, except_col=['permno', 'date', 'mktrf', 'rf', 'shrout'])

Original maximum of ret: 19.0 and Original minimum of ret: -0.971698

New maximum of ret: 0.130435 and New minimum of ret: -0.111111
 

Original maximum of retx: 19.0 and Original minimum of retx: -0.971698

New maximum of retx: 0.130435 and New minimum of retx: -0.111111
 

Original maximum of prc: 149200.0 and Original minimum of prc: 0.0039

New maximum of prc: 103.0 and New minimum of prc: 0.25
 

Original maximum of exret: 18.99999 and Original minimum of exret: -0.971908

New maximum of exret: 0.130195 and New minimum of exret: -0.111321
 



In [120]:
daily_data

,permno,date,mktrf,rf,shrout,ret,retx,prc,exret
1,10000,1986-01-08,-0.0216,0.00025,3680.0,-0.024390,-0.024390,2.500,-0.024640
2,10000,1986-01-09,-0.0117,0.00025,3680.0,0.000000,0.000000,2.500,-0.000250
3,10000,1986-01-10,-0.0002,0.00025,3680.0,0.000000,0.000000,2.500,-0.000250
4,10000,1986-01-13,0.0028,0.00025,3680.0,0.050000,0.050000,2.625,0.049750
5,10000,1986-01-14,0.0001,0.00025,3680.0,0.047619,0.047619,2.750,0.047369
...,...,...,...,...,...,...,...,...,...
83643732,93436,2012-12-24,-0.0024,0.00001,113779.0,0.008235,0.008235,34.280,0.008225
83643733,93436,2012-12-26,-0.0054,0.00001,113779.0,-0.020128,-0.020128,33.590,-0.020138
83643734,93436,2012-12-27,-0.0011,0.00001,113779.0,0.002977,0.002977,33.690,0.002967
83643735,93436,2012-12-28,-0.0100,0.00001,113779.0,-0.013951,-0.013951,33.220,-0.013961


Four monthly estimation periods.   

Using 1,2,3, and 5 years requiring 10,20,24 and 24 valid montlhy excess returns observations.   

24 monthly data points caculate beta for 5 year follows common prcatiece when using monthly data to estimate beta.   

We denote these estimates of market beta, caculated using monthly excess return data, as $\beta^{kY}$, where $k \in \{1,2,3,5\}$.   

The daily stock return data come from __crsp.msf__, and monthly MKT factor returns and risk free rates come from Kenneth French's data library.

In [85]:
# Data description for the CRSP monthly stock file
conn.describe_table(library='crsp', table='msf')

Approximately 5037353 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),CUSIP Header
1,permno,True,INTEGER,PERMNO
2,permco,True,INTEGER,PERMCO
3,issuno,True,INTEGER,Nasdaq Issue Number
4,hexcd,True,SMALLINT,Exchange Code Header
5,hsiccd,True,INTEGER,Standard Industrial Classification Code
6,date,True,DATE,Date of Observation
7,bidlo,True,"NUMERIC(11, 5)",Bid or Low Price
8,askhi,True,"NUMERIC(11, 5)",Ask or High Price
9,prc,True,"NUMERIC(11, 5)",Price or Bid/Ask Average


In [86]:
# Data description for fama french factors daily
conn.describe_table(library='ff', table='factors_monthly')

Approximately 1176 rows in ff.factors_monthly.


,name,nullable,type,comment
0,date,True,DATE,Date (SAS). Eventus Format - First day of the ...
1,mktrf,True,"NUMERIC(8, 6)",Excess Return on the Market
2,smb,True,"NUMERIC(8, 6)",Small-Minus-Big Return
3,hml,True,"NUMERIC(8, 6)",High-Minus-Low Return
4,rf,True,"NUMERIC(7, 5)",Risk-Free Return Rate (One Month Treasury Bill...
5,year,True,DOUBLE PRECISION,Year
6,month,True,DOUBLE PRECISION,Month
7,umd,True,"NUMERIC(8, 6)",Momentum Factor
8,dateff,True,DATE,Date (SAS). Last Trading Day of the Month


In [ ]:
crsp_m = conn.raw_sql("""
                      SELECT permno, date, ret, retx, prc, shrout
                      FROM crsp.msf
                      WHERE date <= '2012-12-31'
                      """)

In [ ]:
# Download the data from fama french factors daily
ff_m = conn.raw_sql("""
                    SELECT date, mktrf, rf
                    FROM ff.factors_monthly
                    WHERE date <= '2012-12-31'
                    """)

##### using the package function from nafitools

In [ ]:
# Later Updated

#### Cleansing data

In [29]:
test_data = daily_data[daily_data['date'] > '2005-01-01'][['permno', 'date', 'ret', 'mktrf']]

#### CAPM beta, Market beta (Fama and Macbeth, 1973)

In [209]:
def cal_beta(data, date, id, ret, mkt, period, only_cal=False):
    """
    Calculate rolling beta, alpha, and their t-values for each group in the dataset.
    
    Parameters:
    data (pd.DataFrame): The dataset containing the required columns.
    date (str): The date column in the dataset.
    id (str): The column
    ret (str): The column 
    mkt (str): The column 
    period (str): The rolling window period. Choose from '1M', '3M', '6M', '12M', '24M'.
    dataframe (bool): Whether to return the updated dataframe with the calculated values. Defaults to False. 
                        If True, the function will return the updated dataframe. 
                        If False, the function will return a list of dataframes.
    
    Returns:
        pd.DataFrame or list: The updated dataframe with the calculated values or a list of dataframes containing the calculated values
    """
    
    # Define rolling window size and minimum valid days based on period
    period_mapping = {
        '1M': {'window_size': 21, 'min_valid_days': 15},
        '3M': {'window_size': 63, 'min_valid_days': 50},
        '6M': {'window_size': 126, 'min_valid_days': 100},
        '12M': {'window_size': 252, 'min_valid_days': 200},
        '24M': {'window_size': 504, 'min_valid_days': 450}
    }
    
    if period not in period_mapping:
        raise ValueError(f"Invalid period. Please choose from {list(period_mapping.keys())}")
    
    window_size = period_mapping[period]['window_size']
    min_valid_days = period_mapping[period]['min_valid_days']
    
    results_dict = {}
    
    for f_id, firm in data.groupby(id):
        firm_len = len(firm)
        if firm_len < min_valid_days:
            print(f"Skipping {f_id} due to insufficient data (size: {firm_len})")
            continue  # If the data is less than the minimum requirement, skip it.

        # When data length is less than window_size, adjust window_size to data length
        actual_window_size = min(firm_len, window_size)

        X = sm.add_constant(firm[mkt])
        y = firm[ret]
        
        model = RollingOLS(y, X, actual_window_size, min_nobs=min_valid_days)
        results = model.fit()
        
        firm['beta'] = results.params[mkt]
        firm['beta_se'] = results.bse[mkt]
        firm['beta_t'] = results.tvalues[mkt] 
        firm['alpha'] = results.params['const']
        firm['alpha_se'] = results.bse['const']
        firm['alpha_t'] = results.tvalues['const']
        
        results_dict[f_id] = firm[['beta', 'beta_se', 'beta_t', 'alpha', 'alpha_se', 'alpha_t']]
   
    
    if only_cal == True:
        return results_dict
    
    if results_dict:
        results_df = pd.concat(results_dict)
        return results_df

In [210]:
result = cal_beta(test_data, 'date', 'permno', 'ret', 'mktrf', '12M', False)

Skipping 10012 due to insufficient data (size: 147)
Skipping 10089 due to insufficient data (size: 39)
Skipping 10108 due to insufficient data (size: 154)
Skipping 10453 due to insufficient data (size: 166)
Skipping 10691 due to insufficient data (size: 82)
Skipping 10906 due to insufficient data (size: 40)
Skipping 10935 due to insufficient data (size: 165)
Skipping 11216 due to insufficient data (size: 168)
Skipping 11262 due to insufficient data (size: 79)
Skipping 11358 due to insufficient data (size: 167)
Skipping 11371 due to insufficient data (size: 19)
Skipping 11393 due to insufficient data (size: 168)
Skipping 11473 due to insufficient data (size: 121)
Skipping 11483 due to insufficient data (size: 73)
Skipping 11485 due to insufficient data (size: 61)
Skipping 11643 due to insufficient data (size: 54)
Skipping 11646 due to insufficient data (size: 39)
Skipping 11794 due to insufficient data (size: 133)
Skipping 11803 due to insufficient data (size: 118)
Skipping 11851 due to

In [215]:
result

beta   beta_se     beta_t     alpha  alpha_se   alpha_t
10001 5182           NaN       NaN        NaN       NaN       NaN       NaN
      5183           NaN       NaN        NaN       NaN       NaN       NaN
      5184           NaN       NaN        NaN       NaN       NaN       NaN
      5185           NaN       NaN        NaN       NaN       NaN       NaN
      5186           NaN       NaN        NaN       NaN       NaN       NaN
...                  ...       ...        ...       ...       ...       ...
93436 81943088  1.165372  0.109342  10.658055  0.000528  0.001661  0.318069
      81943089  1.165558  0.109162  10.677312  0.000488  0.001658  0.294279
      81943090  1.161237  0.108332  10.719208  0.000352  0.001648  0.213315
      81943091  1.159306  0.107633  10.770895  0.000503  0.001639  0.306704
      81943092  1.159540  0.107610  10.775420  0.000474  0.001639  0.288929

[11801483 rows x 6 columns]

#### Scholes and Williams (1977)'s beta

Scholes and Williams (1977) present evidence that non-synchronous trading can affect empirical estimates of beta using the standard CAPM model.   

To account for nonsynchronous trading, they suggest:   

$$ r_{i,t} = \alpha_i + \beta_{i}^{-} MKT_{t-1} + e_{i,t}^{-}$$
$$ r_{i,t} = \alpha_i + \beta_{i} MKT_{t-1} + e_{i,t}$$
$$r_{i,t} = \alpha_i + \beta_{i}^{+} MKT_{t-1} + e_{i,t}^{+}$$
and define beta as
$$ \beta_{i}^{SW} = \frac{\hat{b}_{i}^{-} + \hat{b}_{i} + \hat{b}_{i}^{+}}{1 + 2 \rho}$$

where $\rho$ is the first-order serial correlation of the market porfolio's excess returns, and $\hat{b}_{i}^{-}$, $\hat{b}_{i}$, and $\hat{b}_{i}^{+}$ are the estimated slope coefficients from regression models.

For Scholes and Williams (1977), beta is best implemented using daily stock return data because monthly return data are less affected by problems caused by non-synchronous trading.   

In this chapter, we use length of sample to calculate Scholes and Williams (1997) as 1 year __daily data__.   

In [ ]:
def cal_beta_sw(data, date, id, ret, mkt, period, dataframe=False):
    """
    Calculate Scholes and Williams adjusted beta, its t-value, and corresponding alpha with its t-value.
    
    Parameters:
    data (pd.DataFrame): The dataset containing the required columns.
    date (str): The date column in the dataset.
    id (str): The column representing the unique identifier for each asset (e.g., stock ID).
    ret (str): The column representing the asset's returns.
    mkt (str): The column representing the market's returns.
    period (str): The rolling window period. Choose from '1M', '3M', '6M', '12M', '24M'.
    dataframe (bool): Whether to return the updated dataframe with the calculated values. Defaults to False.
    
    Returns:
        pd.DataFrame or list: The updated dataframe with the calculated values or a list of dataframes containing the calculated values
    """
    # Define rolling window size and minimum valid days based on period
    period_mapping = {
        '1M': {'window_size': 21, 'min_valid_days': 15},
        '3M': {'window_size': 63, 'min_valid_days': 50},
        '6M': {'window_size': 126, 'min_valid_days': 100},
        '12M': {'window_size': 252, 'min_valid_days': 200},
        '24M': {'window_size': 504, 'min_valid_days': 450}
    }
    
    if period not in period_mapping:
        raise ValueError(f"Invalid period. Please choose from {list(period_mapping.keys())}")
    
    window_size = period_mapping[period]['window_size']
    min_valid_days = period_mapping[period]['min_valid_days']


    print("Calculating beta_sw \n")
    
    print("Calculating beta_{t-1} \n")
    beta_tm1 = cal_beta(data.shift(-1), date, id, ret, mkt, period, dataframe=True) # 전체인데?
    print("-----------------------------------")
    print("Calculating beta_t \n")
    beta_t = cal_beta(data, date, id, ret, mkt, period, dataframe=True)
    print("-----------------------------------")
    print("Calculating beta_{t+1} \n")
    beta_tp1 = cal_beta(data.shift(1), date, id, ret, mkt, period, dataframe=True)
    
    rho_model = AutoReg(data[mkt], lags=1).fit()
    rho = rho_model.params[1]
    
    results_list = []

    for idx in range(len(beta_t)):
        # Calculate beta_sw
        beta_sum = beta_tm1[idx]['beta'] + beta_t[idx]['beta'] + beta_tp1[idx]['beta']
        beta_sw = beta_sum / (1 + 2 * rho)
        
        # Calculate standard error for beta_sw using the correct formula
        beta_se_tm1 = beta_tm1[idx]['beta_se']  # assuming beta_se is the standard error from the previous beta calculation
        beta_se_t = beta_t[idx]['beta_se']
        beta_se_tp1 = beta_tp1[idx]['beta_se']
        
        # Assuming the covariance between beta_{-1} and beta_{+1} is zero for simplicity
        beta_se_sw = np.sqrt(beta_se_t**2 + 0.25 * beta_se_tm1**2 + 0.25 * beta_se_tp1**2) / (1 + 2 * rho)
        beta_sw_t = beta_sw / beta_se_sw
        
        # Calculate alpha using beta_sw
        firm_len = len(beta_t[idx])
        if firm_len < min_valid_days:
            print(f"Skipping {f_id} due to insufficient data (size: {firm_len})")
            continue  # If the data is less than the minimum requirement, skip it.

        # When data length is less than window_size, adjust window_size to data length
        actual_window_size = min(firm_len, window_size)
        
        
        X_sw = sm.add_constant(data[data['permno'] == beta_t[idx].index]['mkt'])
        y_sw = data[data['permno'] == beta_t[idx].index]['ret']
        
        model_sw = RollingOLS(y_sw, X_sw, actual_window_size, min_nobs=min_valid_days)
        results_sw = model_sw.fit()
        
        alpha_sw = results_sw.params['const']
        alpha_sw_t = results_sw.tvalues['const']

        # Create a dataframe for the results
        sw_result = pd.DataFrame({
            date: beta_t[idx][date],
            'beta_sw': beta_sw,
            'beta_sw_t': beta_sw_t,
            'alpha_sw': alpha_sw,
            'alpha_sw_t': alpha_sw_t
        })

        results_list.append(sw_result)
    
    if dataframe:
        if results_list:
            results_df = pd.concat(results_list)
            data.update(results_df)
        return data
    
    return results_list

In [179]:
len(result)

9377

In [183]:
result[0]

,beta,beta_t,alpha,alpha_t
5182,NaN,NaN,NaN,NaN
5183,NaN,NaN,NaN,NaN
5184,NaN,NaN,NaN,NaN
5185,NaN,NaN,NaN,NaN
5186,NaN,NaN,NaN,NaN
...,...,...,...,...
6951,0.186605,4.924172,0.000451,0.784065
6952,0.186586,4.924340,0.000455,0.791009
6953,0.182207,4.782522,0.000504,0.870444
6954,0.182884,4.803584,0.000520,0.897247


In [175]:
def cal_beta_sw(data, date, id, ret, mkt, period, dataframe=False):
    """
    Calculate Scholes and Williams adjusted beta, its t-value, and corresponding alpha with its t-value.
    
    Parameters:
    data (pd.DataFrame): The dataset containing the required columns.
    date (str): The date column in the dataset.
    id (str): The column representing the unique identifier for each asset (e.g., stock ID).
    ret (str): The column representing the asset's returns.
    mkt (str): The column representing the market's returns.
    period (str): The rolling window period. Choose from '1M', '3M', '6M', '12M', '24M'.
    dataframe (bool): Whether to return the updated dataframe with the calculated values. Defaults to False.
    
    Returns:
        pd.DataFrame or list: The updated dataframe with the calculated values or a list of dataframes containing the calculated values
    """
    
    print("Calculating beta_sw \n")
    
    print("Calculating beta_{t-1} \n")
    beta_tm1 = cal_beta(data.shift(-1), date, id, ret, mkt, period, dataframe=False)
    print("-----------------------------------")
    print("Calculating beta_t \n")
    beta_t = cal_beta(data, date, id, ret, mkt, period, dataframe=False)
    print("-----------------------------------")
    print("Calculating beta_{t+1} \n")
    beta_tp1 = cal_beta(data.shift(1), date, id, ret, mkt, period, dataframe=False)
    
    rho_model = AutoReg(data[mkt], lags=1).fit()
    rho = rho_model.params[1]
    
    results_list = []

    for idx in range(len(beta_t)):
        beta_sum = beta_tm1[idx]['beta'] + beta_t[idx]['beta'] + beta_tp1[idx]['beta']
        beta_sw = beta_sum / (1 + 2 * rho)
        
        # Calculate standard error for beta_sw
        se_beta_tm1 = beta_tm1[idx]['beta_t'] / beta_tm1[idx]['beta']
        se_beta_t = beta_t[idx]['beta_t'] / beta_t[idx]['beta']
        se_beta_tp1 = beta_tp1[idx]['beta_t'] / beta_tp1[idx]['beta']
        
        se_beta_sw = np.sqrt(se_beta_tm1**2 + se_beta_t**2 + se_beta_tp1**2) / (1 + 2 * rho)
        beta_sw_t = beta_sw / se_beta_sw
        #
        # Calculate alpha using beta_sw
        X_sw = sm.add_constant(data[data['permno'] == beta_t[idx].index]['mkt'])
        y_sw = data[data['permno'] == beta_t[idx].index]['ret']
        
        model_sw = sm.OLS(y_sw, X_sw)
        results_sw = model_sw.fit()
        
        alpha_sw = results_sw.params['const']
        alpha_sw_t = results_sw.tvalues['const']

        # Create a dataframe for the results
        sw_result = pd.DataFrame({
            date: beta_t[idx][date],
            'beta_sw': beta_sw,
            'beta_sw_t': beta_sw_t,
            'alpha_sw': alpha_sw,
            'alpha_sw_t': alpha_sw_t
        })

        results_list.append(sw_result)
    
    if dataframe:
        if results_list:
            results_df = pd.concat(results_list)
            data.update(results_df)
        return data
    
    return results_list

In [176]:
sw_beta = cal_beta_sw(test_data, 'date', 'permno', 'ret', 'mktrf', '12M', True)

Calculating beta_sw 

Calculating beta_{t-1} 

Skipping 10012.0 due to insufficient data (size: 147)
Skipping 10089.0 due to insufficient data (size: 39)
Skipping 10108.0 due to insufficient data (size: 154)
Skipping 10453.0 due to insufficient data (size: 166)
Skipping 10691.0 due to insufficient data (size: 82)
Skipping 10906.0 due to insufficient data (size: 40)
Skipping 10935.0 due to insufficient data (size: 165)
Skipping 11216.0 due to insufficient data (size: 168)
Skipping 11262.0 due to insufficient data (size: 79)
Skipping 11358.0 due to insufficient data (size: 167)
Skipping 11371.0 due to insufficient data (size: 19)
Skipping 11393.0 due to insufficient data (size: 168)
Skipping 11473.0 due to insufficient data (size: 121)
Skipping 11483.0 due to insufficient data (size: 73)
Skipping 11485.0 due to insufficient data (size: 61)
Skipping 11643.0 due to insufficient data (size: 54)
Skipping 11646.0 due to insufficient data (size: 39)
Skipping 11794.0 due to insufficient data (s

c:\Users\YeonChan Kang\anaconda3\envs\fdb\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\YeonChan Kang\AppData\Local\Temp\ipykernel_155632\2447871842.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rho = rho_model.params[1]


KeyError: 'mktrf'

#### Dimson (1979) beta

Dimson (1979) shows that when a stock is infrequently traded, estimating of beta using the CAPM model can be severely biased.   

Dimson(1979) beta is defined as:
$$ \beta_{i}^{D} = \sum_{k=-5}^{k=5} \hat{b}_{i}^k$$
where the $$\beta_{i}^{k}$$ are the estimated slope coefficients from the regression model:
$$ r_{i,t} = a_i + \sum_{k=-5}^{k=5} \beta_{i}^{k} MKT_{t+k} + e_{i,t}

In this chapter, we calculate $\beta^D$ for stock $i$ in __month__ $t$ using one year's worth of daily return data for t-11 through t.

In [167]:
def calculate_dimson_beta(data, id, ret, mkt, period, lag_range=5, dataframe=False):
    """
    Calculate Dimson Beta by summing the betas over a range of lags.
    """
    dimson_betas = []

    for lag in range(-lag_range, lag_range + 1):
        # Shift the market return by the lag
        data[f'{mkt}_lag_{lag}'] = data.groupby(id)[mkt].shift(-lag)
        
        # Calculate beta for the shifted data
        beta_results = cal_beta(data, id, ret, f'{mkt}_lag_{lag}', period, dataframe=dataframe)
        
        # Append the calculated betas to the list
        dimson_betas.append(beta_results[['beta']])
    
    # Sum the betas across all lags to get Dimson Beta
    dimson_beta = sum(dimson_betas)
    data['dimson_beta'] = dimson_beta
    
    return data
    

# 2. Summary Statistics

As same as winsorizing we skip the explanation of functions to caculate Summary Statistics.

For more on this, see Ch2. Summary Statistics

126p